# 데이터 전처리
Colab의 Local disk의 여유 공간은 약 30~60GB 정도로 AI Hub에 있는 데이터를 풀어내기에는 적합하지 않다. Local에 있는 Disk를 이용하여 Resize를 수행하기로 한다.

## 1.다운로드 데이터 분석
AI-Hub에서 다운로드 받은 데이터는 아래와 같은 구조를 가진다. 


```bash
랜드마크 이미지
├─Training
│  └─세종특별자치시
│          [라벨]세종특별자치시.zip
│          [원천]세종특별자치시_001.zip
└─Validation
    └─세종특별자치시
            [라벨]세종특별자치시.zip
            [원천]세종특별자치시_001.zip
```
테스트와 검증 데이터가 분리되어 있으니 그대로 이용하고, [라벨] 파일은 metadata가 있는 json파일이다. 현 시점에서는 필요없으니 무시한다. 즉, [원천] 파일만 압축을 풀어 Resize를 수행하면 된다.


## 2.데이터 압축 해제
중간폴더의 지역명은 필요없으니 무시하고 디렉토리를 구성하여 압축을 해제한다.

In [25]:
import os, glob

base_dir = '/content/랜드마크 이미지/'
extract_dir = '/content/extracts'

for dtype in os.listdir(base_dir):
    dst_dir = os.path.join(extract_dir, dtype)
    os.makedirs(dst_dir, exist_ok=True)
    for file_name in glob.glob(os.path.join(base_dir, dtype) + '/**/*원천*', recursive=True):
        file_name = '\'' + file_name + '\''
        !unzip -o -qq {file_name} -d {dst_dir}

## 3.RESIZE
압축 해제한 데이터의 구조는 아래와 같다. 파일 확장자명은 JPG or jpg로 되어 있으니 해당 파일을 찾아
OpenCV로 0.1배로 Resize하여 저장한다.

```bash
6.25격전지개미고개
  └─6.25격전지개미고개_001_40613647.JPG
    6.25격전지개미고개_003_40613648.JPG
    6.25격전지개미고개_004_40613649.JPG
    6.25격전지개미고개_005_40613650.JPG
    6.25격전지개미고개_006_40613651.JPG
    (...)
```

In [ ]:
from tqdm.notebook import tqdm

resized_dir = '/content/resizeds'
for dtype in os.listdir(resized_dir):
    dst_dir = os.path.join(resized_dir, dtype)
    os.makedirs(dst_dir, exist_ok=True)
    src_list = glob.glob(os.path.join(extract_dir, dtype) + '/**/*.{JPG, jpg}')
    print(src_list[:5])
    
    
    


In [ ]:
region_list = ['세종특별자치시']
dataset_dir = '/datasets'
project_name = 'Landmark_Images'
base_dir = os.path.join(dataset_dir, project_name)

print(base_dir)

/datasets/Landmark_Images


In [ ]:
!apt-get install unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [ ]:
extract_dir = os.path.join(base_dir, 'extracts')
os.mkdir(extract_dir)
for data_type in ['Training', 'Validation']:
    data_type_dir = os.path.join(extract_dir, data_type)
    if not os.path.isdir(data_type_dir):
        os.mkdir(data_type_dir)

    for region in region_list:
        region_dir = os.path.join(data_type_dir, region)
        if not os.path.isdir(region_dir):
            os.mkdir(region_dir)
        src_file = os.path.join(base_dir, 'downloads', data_type, region, "[원천]{}_001.zip".format(region))
        !unzip -qq {src_file} -d {region_dir}

In [ ]:
import cv2

In [ ]:
extract_dir = os.path.join(base_dir, 'extracts')
resized_dir = os.path.join(base_dir, 'resizeds')
os.mkdir(resized_dir)

In [ ]:
from tqdm.notebook import tqdm

for data_type in ['Training', 'Validation']:
    data_type_dir = os.path.join(resized_dir, data_type)
    if not os.path.isdir(data_type_dir):
        os.mkdir(data_type_dir)
    
    for region in region_list:
        region_dir = os.path.join(extract_dir, data_type, region)
        cls_list = os.listdir(region_dir)
        
        for cls in cls_list:
            target_dir = os.path.join(resized_dir, data_type, cls)
            if not os.path.isdir(target_dir):
                os.mkdir(target_dir)

            print("Resize class: {}, data_type: {}".format(cls, data_type))
            for fname in tqdm(os.listdir(os.path.join(region_dir, cls))):
                img = cv2.imread(os.path.join(region_dir, cls, fname))

                resized_img = cv2.resize(img, (0, 0), fx=0.1, fy=0.1, interpolation=cv2.INTER_AREA)

                cv2.imwrite(os.path.join(target_dir, fname), resized_img)



            




        # src_file = os.path.join(base_dir, 'downloads', data_type, region, "[원천]{}_001.zip".format(region))

Resize class: 6.25격전지개미고개, data_type: Training


  0%|          | 0/153 [00:00<?, ?it/s]

Resize class: BRT작은미술관, data_type: Training


  0%|          | 0/88 [00:00<?, ?it/s]

Resize class: 가락마을20단지호반베르디움5차아파트, data_type: Training


...

  0%|          | 0/23 [00:00<?, ?it/s]

Resize class: 힐스테이트세종리버파크7단지, data_type: Validation


  0%|          | 0/13 [00:00<?, ?it/s]

Resize class: 힐스테이트세종리버파크8단지, data_type: Validation


  0%|          | 0/13 [00:00<?, ?it/s]